In [ ]:
import pandas as pd

# Load Millburn data
df = pd.read_excel('../raw_data/16618 - Millburn Absence History 7.1.20 to 6.30.25.xlsx')

print(f'Rows: {len(df)}')
print(f'Columns: {len(df.columns)}')print()


Rows: 94538
Columns: 13


In [ ]:
print('Column names:', list(df.columns))
print()
print(f'Employee Type - unique values: {df["Employee Type"].nunique()}')
print(df['Employee Type'].unique())

Column names: ['School Year', 'Date', 'Reason', 'Employee Identifier', 'Hire Date', 'Employee Title', 'Employee Type', 'Start Time', 'End Time', 'Duration', 'Absence Type', 'Filled', 'Needs Substitute']

Employee Type - unique values: 52
['Bus Driver' 'Transportation' 'Coordinator' 'Custodian'
 'Paraprofessional' 'Lunch Aide' 'Secretary10' 'Supervisor B&G'
 'Maint/Grounds' 'Registrar' 'Teacher' 'Secretary12' 'Teacher SpecEd'
 'Business Office' 'Principal' 'Principal-3' 'Secretary12 Business'
 'Secretary12 Supt' 'Secretary12 HR' 'Secretary12 SpclSvcs'
 'Director Security' 'Security Staff' 'Special Services' 'Superintendent'
 'Supervisor Guidance' 'Supervisor Music' 'Instructional Supvsr'
 'Principal-2' 'Teacher-tech facil/librarian/MS related arts'
 'Supervisor- ELA' 'Director Athletic' 'Teacher Music' 'Technology'
 'Payroll Coord' 'Secretary10 SpclSvcs' 'Supp Cust' 'Secretary12 Trans'
 'Asst Superintendent' 'Business Admin' 'Supervisor SpclSvcs'
 'Director Technology' 'Director' 'Direc

In [3]:
# Apply all cleaning rules (same as agents)
df_clean = df.copy()

# Rule 1: Remove Unfilled + NO Substitute
df_clean = df_clean[~((df_clean['Filled'] == 'Unfilled') & (df_clean['Needs Substitute'] == 'NO'))]
print(f'After Rule 1 (Unfilled+NO): {len(df_clean)}')

# Rule 2: Keep only Teacher, Teacher Music, Teacher SpecEd
df_clean = df_clean[df_clean['Employee Type'].isin(['Teacher', 'Teacher Music', 'Teacher SpecEd'])]
print(f'After Rule 2 (Employee Type): {len(df_clean)}')

# Rule 3: Date must fall within School Year (July 1 - June 30)
def date_in_school_year(row):
    sy = str(row['School Year']).split('-')
    if len(sy) != 2: return True
    start_year, end_year = int(sy[0]), int(sy[1])
    start = pd.Timestamp(year=start_year, month=7, day=1)
    end = pd.Timestamp(year=end_year, month=6, day=30)
    dt = pd.to_datetime(row['Date'])
    return start <= dt <= end
df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
df_clean = df_clean[df_clean.apply(date_in_school_year, axis=1)]
print(f'After Rule 3 (School Year dates): {len(df_clean)}')

# Absence Days: Full Day=1.0, AM/PM Half Day=0.5, Custom=Duration/7.5
def calc_absence_days(row):
    t = str(row.get('Absence Type', '')).strip()
    if t == 'Full Day': return 1.0
    if t in ['AM Half Day', 'PM Half Day']: return 0.5
    if t == 'Custom Duration': return pd.to_numeric(row.get('Duration', 0), errors='coerce') / 7.5 or 0
    return 0
df_clean['Absence_Days'] = df_clean.apply(calc_absence_days, axis=1)
print(f'Final rows: {len(df_clean)}')

After Rule 1 (Unfilled+NO): 51127
After Rule 2 (Employee Type): 39948
After Rule 3 (School Year dates): 39948
Final rows: 39948


In [ ]:
df_clean.head()

,School Year,Date,Reason,Employee Identifier,Hire Date,Employee Title,Employee Type,Start Time,End Time,Duration,Absence Type,Filled,Needs Substitute,Absence_Days
8508,2021-2022,2021-10-25,Self,4057,2015-08-13 18:45:38,Grade 3 teacher,Teacher,08:20:00,15:50:00,7.500000,Full Day,Unfilled,YES,1.0
8509,2021-2022,2021-11-17,Self,4057,2015-08-13 18:45:38,Grade 3 teacher,Teacher,08:20:00,15:50:00,7.500000,Full Day,Unfilled,YES,1.0
8510,2021-2022,2021-11-24,Self,4057,2015-08-13 18:45:38,Grade 3 teacher,Teacher,08:20:00,15:50:00,7.500000,Full Day,Unfilled,YES,1.0
8511,2021-2022,2021-12-15,Self,4057,2015-08-13 18:45:38,Grade 3 teacher,Teacher,11:40:00,15:50:00,4.166666,PM Half Day,Filled,YES,0.5
8512,2021-2022,2021-12-15,Self,4057,2015-08-13 18:45:38,Grade 3 teacher,Teacher,08:20:00,11:40:00,3.333333,AM Half Day,Unfilled,YES,0.5


In [8]:
# Premium calculation (same logic as app)
DEDUCTIBLE = 20
CC_DAYS = 60
REPLACEMENT_COST = 150.0
ARK_RATE = 0.15
ABCOVER_RATE = 0.15
CC_MAX = DEDUCTIBLE + CC_DAYS

# Aggregate Total_Days per teacher (across all school years)
teacher_days = df_clean.groupby('Employee Identifier')['Absence_Days'].sum().reset_index()
teacher_days.columns = ['Employee Identifier', 'Total_Days']
total_days_per_teacher = teacher_days.set_index('Employee Identifier')['Total_Days']

# Staff in CC range (days > deductible AND <= cc_max)
staff_cc = total_days_per_teacher[(total_days_per_teacher > DEDUCTIBLE) & (total_days_per_teacher <= CC_MAX)]
# Total CC days: for each teacher, count (days - deductible) capped at (cc_max - deductible)
total_cc_days = sum(min(d - DEDUCTIBLE, CC_MAX - DEDUCTIBLE) for d in staff_cc)

# High claimants (days > cc_max)
staff_high = total_days_per_teacher[total_days_per_teacher > CC_MAX]
excess_days = (staff_high - CC_MAX).sum()

# Premium
replacement_cost_cc = REPLACEMENT_COST * total_cc_days
ark_commission = replacement_cost_cc * ARK_RATE
abcover_commission = replacement_cost_cc * ABCOVER_RATE
total_premium = replacement_cost_cc + ark_commission + abcover_commission

print(f'Teachers: {len(teacher_days)} | In CC range: {len(staff_cc)} | High claimants: {len(staff_high)}')
print(f'Total CC Days: {total_cc_days:.2f} | Excess Days: {excess_days:.2f}')
print(f'Replacement (CC): ${replacement_cost_cc:,.2f} | ARK: ${ark_commission:,.2f} | ABCover: ${abcover_commission:,.2f}')
print(f'TOTAL PREMIUM: ${total_premium:,.2f}')

Teachers: 527 | In CC range: 305 | High claimants: 60
Total CC Days: 6617.57 | Excess Days: 18623.46
Replacement (CC): $992,635.67 | ARK: $148,895.35 | ABCover: $148,895.35
TOTAL PREMIUM: $1,290,426.37


In [ ]:
# School Year Metrics table (same format as app)
REPLACEMENT_COST = 150.0
SCHOOL_YEAR_DAYS = 180
DEDUCTIBLE = 20
CC_DAYS = 60

table_data = []
for sy in sorted(df_clean['School Year'].dropna().unique()):
    sy_data = df_clean[df_clean['School Year'] == sy]
    total_staff = sy_data['Employee Identifier'].nunique()
    total_absences = sy_data['Absence_Days'].sum()
    total_rc = total_absences * REPLACEMENT_COST
    table_data.append({
        'School Year': sy,
        'Total # Of Staff': total_staff,
        'Total # of Absences': f'{total_absences:,.2f}',
        'Replacement Cost Per Day ($)': f'${REPLACEMENT_COST:.2f}',
        'Total Replacement Cost to District ($)': f'${total_rc:,.2f}',
        'Amt. of School Year Days': SCHOOL_YEAR_DAYS,
        "Waiting Period 'Deductible' (Days)": DEDUCTIBLE,
        "'CC' Maximum Days (per staff member)": CC_DAYS
    })

# 5-Yr Avg row
n = len(table_data)
if n > 1:
    avg_staff = sum(df_clean.groupby('School Year')['Employee Identifier'].nunique()) / n
    avg_absences = df_clean['Absence_Days'].sum() / n
    avg_rc = avg_absences * REPLACEMENT_COST
    table_data.append({
        'School Year': '5-Yr Avg',
        'Total # Of Staff': f'{avg_staff:.1f}',
        'Total # of Absences': f'{avg_absences:,.1f}',
        'Replacement Cost Per Day ($)': f'${REPLACEMENT_COST:.2f}',
        'Total Replacement Cost to District ($)': f'${avg_rc:,.2f}',
        'Amt. of School Year Days': SCHOOL_YEAR_DAYS,
        "Waiting Period 'Deductible' (Days)": DEDUCTIBLE,
        "'CC' Maximum Days (per staff member)": CC_DAYS
    })

pd.DataFrame(table_data)  # School Year Metrics (same as app)

,School Year,Total # Of Staff,Total # of Absences,Replacement Cost Per Day ($),Total Replacement Cost to District ($),Amt. of School Year Days,Waiting Period 'Deductible' (Days),'CC' Maximum Days (per staff member)
0,2020-2021,237,"6,065.53",$150.00,"$909,830.00",180,20,60
1,2021-2022,396,"8,261.71",$150.00,"$1,239,257.00",180,20,60
2,2022-2023,406,"8,753.11",$150.00,"$1,312,966.67",180,20,60
3,2023-2024,417,"7,510.92",$150.00,"$1,126,638.33",180,20,60
4,2024-2025,407,"6,939.36",$150.00,"$1,040,903.33",180,20,60
5,5-Yr Avg,372.6,"7,506.1",$150.00,"$1,125,919.07",180,20,60


In [ ]:
# Calculation Breakdown (same format as app)
below_deductible = (total_days_per_teacher <= DEDUCTIBLE).sum()
high_claimant_cost = REPLACEMENT_COST * excess_days
num_staff_cc = len(staff_cc)
num_high = len(staff_high)
high_claimant_cost = REPLACEMENT_COST * excess_days

print("=" * 60)
print("CALCULATION BREAKDOWN")
print("=" * 60)
print()
print("1. Base (Replacement Cost × CC Days)")
print(f"   Replacement Cost per Day × Total CC Days = ${REPLACEMENT_COST:,.2f} × {total_cc_days:,.2f} = ${replacement_cost_cc:,.2f}")
print()
print("2. ARK Commission")
print(f"   Base × ARK Rate = ${replacement_cost_cc:,.2f} × {ARK_RATE*100:.0f}% = ${ark_commission:,.2f}")
print()
print("3. ABCover Commission")
print(f"   Base × ABCover Rate = ${replacement_cost_cc:,.2f} × {ABCOVER_RATE*100:.0f}% = ${abcover_commission:,.2f}")
print()
print("4. Total Premium")
print(f"   Base + ARK + ABCover = ${replacement_cost_cc:,.2f} + ${ark_commission:,.2f} + ${abcover_commission:,.2f} = ${total_premium:,.2f}")
print()
print("-" * 60)
print("Teacher Distribution (Inputs to Calculation)")
print("-" * 60)
print(f"  Total Teachers: {len(teacher_days):,}")
print(f"  Below Deductible (≤{DEDUCTIBLE} days): {below_deductible:,}")
print(f"  In CC Range ({DEDUCTIBLE+1}-{CC_MAX} days): {num_staff_cc:,}")
print(f"  High Claimants (>{CC_MAX} days): {num_high:,}")
print()
print("Inputs: Deductible={}, CC Days={}, CC Max={}, Replacement Cost=${}/day".format(DEDUCTIBLE, CC_DAYS, CC_MAX, REPLACEMENT_COST))

CALCULATION BREAKDOWN

1. Base (Replacement Cost × CC Days)
   Replacement Cost per Day × Total CC Days = $150.00 × 6,617.57 = $992,635.67

2. ARK Commission
   Base × ARK Rate = $992,635.67 × 15% = $148,895.35

3. ABCover Commission
   Base × ABCover Rate = $992,635.67 × 15% = $148,895.35

4. Total Premium
   Base + ARK + ABCover = $992,635.67 + $148,895.35 + $148,895.35 = $1,290,426.37

------------------------------------------------------------
Teacher Distribution (Inputs to Calculation)
------------------------------------------------------------
  Total Teachers: 527
  Below Deductible (≤20 days): 162
  In CC Range (21-80 days): 305
  High Claimants (>80 days): 60

Inputs: Deductible=20, CC Days=60, CC Max=80, Replacement Cost=$150.0/day


In [ ]:
# CC Range & High Claimant Staff Details (same as app expanders)
cc_details = [{'employee_id': eid, 'total_days': d, 'days_in_cc_range': min(d - DEDUCTIBLE, CC_MAX - DEDUCTIBLE)} 
              for eid, d in staff_cc.items()]
hc_details = [{'employee_id': eid, 'total_days': d, 'excess_days': d - CC_MAX} for eid, d in staff_high.items()]

from IPython.display import display
print("CC Range Staff Details (How CC Days are Calculated) - Total CC Days:", f"{total_cc_days:,.2f}")
display(pd.DataFrame(cc_details).head(10)) if cc_details else print("  No staff in CC range.")
print()
print("High Claimant Staff Details (Excess Days) - Total Excess:", f"{excess_days:,.2f} | Cost: ${high_claimant_cost:,.2f}")
display(pd.DataFrame(hc_details).head(10)) if hc_details else print("  No high claimant staff.")